In [1]:
print("Hello")

Hello


In [2]:
#DAILY PERCENTAGE RETURNS DATA... need this to see how the price reacts to a given news release
# Remember, we are just doing the daily percentage gains, not going intraday yet, to keep things simple.


#Let's set up our Alpaca API...

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["FB", "TWTR"]

# Set timeframe to one day ("1Day") for the Alpaca API
timeframe = "1Day"

# Get current closing prices for FB and TWTR
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df

# Display sample data
df_portfolio

# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2019-07-14", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# Get closing prices for FB and TWTR from the last year
df_portfolio_year = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

# Display sample data
df_portfolio_year.head(10)


Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2019-07-15 04:00:00+00:00,203.91,205.3300,201.8200,144229,204.25,16972563,203.871252,FB
2019-07-16 04:00:00+00:00,203.84,205.4707,203.1000,107939,203.89,12682328,204.123233,FB
2019-07-17 04:00:00+00:00,201.80,204.3600,201.5900,112850,204.18,13172182,202.498845,FB
2019-07-18 04:00:00+00:00,200.78,202.4700,199.3800,111377,200.15,12719444,200.623803,FB
2019-07-19 04:00:00+00:00,198.36,202.3300,198.0700,102977,202.18,13597320,200.086825,FB
2019-07-22 04:00:00+00:00,202.32,202.5700,198.8100,120169,199.91,14522116,201.301601,FB
2019-07-23 04:00:00+00:00,202.36,204.2420,200.9625,123223,202.84,15469348,202.229324,FB
2019-07-24 04:00:00+00:00,204.66,204.8100,197.2200,271250,197.63,33959225,203.377692,FB
2019-07-25 04:00:00+00:00,200.71,208.6600,198.2600,341775,206.70,41465948,201.530117,FB


In [3]:

# Reorganize the DataFrame using the above data we just got from alpaca 

# Separate ticker data
FB = df_portfolio_year[df_portfolio_year['symbol']=='FB'].drop('symbol', axis=1)
TWTR = df_portfolio_year[df_portfolio_year['symbol']=='TWTR'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_portfolio_year = pd.concat([FB, TWTR],axis=1, keys=['FB','TWTR'])


In [ ]:
# GET GEOSPACIAL DATA
# We are interested in US companies that release 'hot' news.
# Where in the US are these companies located?
# We get their location, and we should be able to translate the "name" of that location into "coordinates"
# Once it is in "coordinates", we can plot that point on the Geo map using HVplot


# Frist step would be to get the locations of each company.

# Step 2 is to get access to a dataset that translates cities/towns in the US to coordinates.

# Lastly we can plot those coordinates on the map.

In [12]:
# STEP 2: FIND DATASET TRANSLATING CITY NAME TO COORDINATES
# We can use this API: "GEOCODING API" https://openweathermap.org/api/geocoding-api 
# Here is a description from their website: 

# "Geocoding API is a simple tool that we have developed to ease the search for locations while working with geographic names and coordinates.
# Supporting API calls by geographical coordinates is the most accurate way to specify any location, that is why this method is integrated in all OpenWeather APIs. 
# However, this way is not always suitable for all users.
# Geocoding is the process of transformation of any location name into geographical coordinates, and the other way around (reverse geocoding).
# OpenWeather’s Geocoding API supports both the direct and reverse methods, working at the level of city names, areas and districts, countries and states:
# Direct geocoding converts the specified name of a location or zip/post code into the exact geographical coordinates;
# Reverse geocoding converts the geographical coordinates into the names of the nearby locations."

# We will use direct geocoding.

import requests
# import requests

# Create variable to hold request url
url = "http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid=628878cc6edbe70ff1805be47e1af1ea"

# Add format specifier to request url
#url = url + "?format=json"

# Execute get request
requests.get(url)


# Execute GET request and store response
response_data = requests.get(url)
# q	required	City name, state code (only for the US) and country code divided by comma. Please use ISO 3166 country codes.
# appid	required	Your unique API key (you can always find it on your account page under the "API key" tab)
# limit	optional	Number of the locations in the API response (up to 5 results can be returned in the API response)

# http://api.openweathermap.org/geo/1.0/direct?q=London&limit=5&appid={API key}

#http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid={3fcc50c01f481adc6307ae95cede2362}

In [13]:
print(response_data.content) #I think I am getting an error because I need to confirm my email. However, that email is taking a while to arrive...

b'[{"name":"Oakland","local_names":{"eo":"Oakland","fa":"\xd8\xa7\xd9\x88\xda\xa9\xd9\x84\xd9\x86\xd8\xaf\xd8\x8c \xda\xa9\xd8\xa7\xd9\x84\xdb\x8c\xd9\x81\xd8\xb1\xd9\x86\xdb\x8c\xd8\xa7","ru":"\xd0\x9e\xd0\xba\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb4","ka":"\xe1\x83\x9d\xe1\x83\x99\xe1\x83\x9a\xe1\x83\x94\xe1\x83\x9c\xe1\x83\x93\xe1\x83\x98","uk":"\xd0\x9e\xd0\xba\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb4","it":"Oakland","en":"Oakland","fr":"Oakland","ja":"\xe3\x82\xaa\xe3\x83\xbc\xe3\x82\xaf\xe3\x83\xa9\xe3\x83\xb3\xe3\x83\x89","cs":"Oakland","ht":"Oakland","kw":"Oakland","mk":"\xd0\x9e\xd1\x83\xd0\xba\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb4","mr":"\xe0\xa4\x93\xe0\xa4\x95\xe0\xa4\xb2\xe0\xa4\x82\xe0\xa4\xa1","lv":"Oklenda","io":"Oakland","ro":"Oakland","mg":"Oakland","no":"Oakland","sw":"Oakland","ml":"\xe0\xb4\x93\xe0\xb4\x95\xe0\xb5\x8d\xe2\x80\x8c\xe0\xb4\xb2\xe0\xb4\xbe\xe0\xb4\xa8\xe0\xb5\x8d\xe0\xb4\xb1\xe0\xb5\x8d, \xe0\xb4\x95\xe0\xb4\xbe\xe0\xb4\xb2\xe0\xb4\xbf\xe0\xb4\xab\xe0\xb5\x8b\xe0\xb5\xbc\xe0

In [22]:
# Code to translate city and state to geocoded data for geo plot

# Initial imports
import pandas as pd
import requests

# Replace 'YOUR_API_KEY' with your actual OpenWeatherMap API key
api_key = '628878cc6edbe70ff1805be47e1af1ea'

# Create an empty list to store the geocoded data
data_list = []

# Create a list of cities and states
cities_and_states = [
    {"city": "New York", "state": "New York"},
    {"city": "Los Angeles", "state": "California"},
    {"city": "Chicago", "state": "Illinois"},
    {"city": "Houston", "state": "Texas"},
    # Add more cities and states as needed
]

for location in cities_and_states:
    city_name = location["city"]
    state_name = location["state"]

    # Use the OpenWeatherMap Geocoding API to fetch latitude and longitude
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name},{state_name}&appid={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        lat = data["coord"]["lat"]
        lon = data["coord"]["lon"]

        # Append the data to the list
        data_list.append({"City": city_name, "State": state_name, "Latitude": lat, "Longitude": lon})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Display the DataFrame
df


,City,State,Latitude,Longitude
0,New York,New York,40.7143,-74.0060
1,Los Angeles,California,34.0522,-118.2437
2,Chicago,Illinois,41.8500,-87.6500
3,Houston,Texas,29.7633,-95.3633
